In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use my GPU
  try:
    tf.config.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2025-05-27 03:12:39.920087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748333559.943151    7734 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748333559.950256    7734 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748333559.968098    7734 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748333559.968118    7734 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748333559.968120    7734 computation_placer.cc:177] computation placer alr

2 Physical GPUs, 1 Logical GPU


I0000 00:00:1748333564.049043    7734 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15252 MB memory:  -> device: 1, name: Tesla P100-PCIE-16GB, pci bus id: 0000:82:00.0, compute capability: 6.0


In [2]:
# load and prepare data for training
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

## Dense Feed Forward Neural Networks (DFNNs)  also known as multi-layer perceptrons(MLPs)
* well suited for inferencing from tabular data

### Construct DFNN model using Sequential API  
* simple interface
* limited functionality

In [3]:
# Sequential API (Very convenient, not very flexible)
modelDFNN1 = keras.Sequential(
    [
        keras.Input(shape=(28 * 28,)),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(10),
    ]
)

modelDFNN1 = keras.Sequential()
modelDFNN1.add(keras.Input(shape=(784,)))
modelDFNN1.add(layers.Dense(512, activation="relu"))
modelDFNN1.add(layers.Dense(256, activation="relu", name="my_layer"))
modelDFNN1.add(layers.Dense(10))

In [4]:
# ToDo #1: List what objects the modelDFNN1 is comprised of (built from)
modelDFNN1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ my_layer (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 535,818 (2.04 MB)

 Trainable params: 535,818 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
modelDFNN1.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

In [6]:
modelDFNN1.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
modelDFNN1.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/5


I0000 00:00:1748333577.862516    8011 service.cc:152] XLA service 0x2aae74002c50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748333577.882878    8011 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-05-27 03:12:59.906518: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748333581.496369    8011 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1748333584.418414    8011 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 - 11s - 6ms/step - accuracy: 0.1498 - loss: 2.3700
Epoch 2/5
1875/1875 - 3s - 2ms/step - accuracy: 0.1469 - loss: 2.3025
Epoch 3/5
1875/1875 - 3s - 2ms/step - accuracy: 0.1478 - loss: 2.3025
Epoch 4/5
1875/1875 - 3s - 2ms/step - accuracy: 0.1490 - loss: 2.3025
Epoch 5/5
1875/1875 - 3s - 2ms/step - accuracy: 0.1503 - loss: 2.3025
313/313 - 1s - 4ms/step - accuracy: 0.1448 - loss: 2.3026


[2.30259108543396, 0.14480000734329224]

In [ ]:
# ToDo #2: What train and test accuracy do you obtain when running for 5 epochs ?  
# training: 0.1503, testing: 0.1448, all very low.

In [7]:
# ToDo #3: Now write code (add code cells to this notebook) to train the model for 20 additional epochs. What train and test accuracy do you obtain? 
modelDFNN1.fit(x_train, y_train, batch_size=32, epochs=20, verbose=2)
modelDFNN1.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/20
1875/1875 - 3s - 2ms/step - accuracy: 0.1511 - loss: 2.3025
Epoch 2/20
1875/1875 - 3s - 1ms/step - accuracy: 0.1547 - loss: 2.3025
Epoch 3/20
1875/1875 - 3s - 2ms/step - accuracy: 0.1515 - loss: 2.3028
Epoch 4/20
1875/1875 - 3s - 1ms/step - accuracy: 0.0904 - loss: 2.3026
Epoch 5/20
1875/1875 - 3s - 2ms/step - accuracy: 0.0904 - loss: 2.3026
Epoch 6/20
1875/1875 - 3s - 2ms/step - accuracy: 0.0904 - loss: 2.3026
Epoch 7/20
1875/1875 - 3s - 1ms/step - accuracy: 0.0904 - loss: 2.3026
Epoch 8/20
1875/1875 - 3s - 1ms/step - accuracy: 0.0904 - loss: 2.3026
Epoch 9/20
1875/1875 - 3s - 2ms/step - accuracy: 0.0904 - loss: 2.3026
Epoch 10/20
1875/1875 - 3s - 2ms/step - accuracy: 0.0904 - loss: 2.3026
Epoch 11/20
1875/1875 - 3s - 2ms/step - accuracy: 0.0904 - loss: 2.3026
Epoch 12/20
1875/1875 - 3s - 2ms/step - accuracy: 0.0904 - loss: 2.3026
Epoch 13/20
1875/1875 - 3s - 2ms/step - accuracy: 0.0904 - loss: 2.3026
Epoch 14/20
1875/1875 - 3s - 2ms/step - accuracy: 0.0904 - loss: 2.3026
E

[2.30259108543396, 0.08919999748468399]

In [8]:
# training: 0.0904, testing: 0.0892, not improving.

### Construct DFNN model using Functional API  
* slightly more complex interface
* additional functionality

In [ ]:
# ToDo #4: Now practice using the OO functional API of keras by constructing your own functional model, modelDFNN2.
# To simplify the task, create this model so that it uses the same architecture as the sequential model.
# For the final Dense layer use activation="softmax"


In [13]:
# Functional API (A bit more flexible)
inputs = keras.Input(shape=((784,)))
x = layers.Dense(512, activation="relu", name="first_layer")(inputs)
x = layers.Dense(256, activation="relu", name="second_layer")(x)
outputs = layers.Dense(10, activation="softmax")(x)
modelDFNN2 = keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
# Now to gain experience invoking methods on objects you have created,
# write code to compile(), fit() your new model on the training data. 
# Then write code to evaluate your new model on the test data. 

In [14]:
modelDFNN2.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

In [15]:
modelDFNN2.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
modelDFNN2.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 4s - 2ms/step - accuracy: 0.9435 - loss: 0.1853
Epoch 2/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9757 - loss: 0.0796
Epoch 3/5
1875/1875 - 3s - 1ms/step - accuracy: 0.9831 - loss: 0.0539
Epoch 4/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9864 - loss: 0.0412
Epoch 5/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9892 - loss: 0.0340
313/313 - 1s - 4ms/step - accuracy: 0.9814 - loss: 0.0646


[0.06457379460334778, 0.9814000129699707]

In [16]:
# ToDo #5. Write down what train and test accuracy do you observe? How does it compare to the sequential model?
# training: 0.9892, testing: 0.9814, it's so much more faster.

In [ ]:
# For fun: explore the architecture space by adding or removing layers from your model, retrain from scratch. 
# Learn how that impacts model performance. State of the art accuracy is around 99.97% . 
# You do not need to achieve that (nor is that expected here) but to give you an idea.

In [17]:
inputs = keras.Input(shape=((784,)))
x = layers.Dense(1024, activation="relu", name="first_layer")(inputs)
x = layers.Dense(512, activation="relu", name="sec_layer")(inputs)
x = layers.Dense(256, activation="relu", name="third_layer")(x)
outputs = layers.Dense(10, activation="softmax")(x)
modelDFNN3 = keras.Model(inputs=inputs, outputs=outputs)

modelDFNN3.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

modelDFNN3.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
modelDFNN3.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 4s - 2ms/step - accuracy: 0.9424 - loss: 0.1897
Epoch 2/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9757 - loss: 0.0793
Epoch 3/5
1875/1875 - 3s - 1ms/step - accuracy: 0.9823 - loss: 0.0562
Epoch 4/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9863 - loss: 0.0428
Epoch 5/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9902 - loss: 0.0309
313/313 - 1s - 4ms/step - accuracy: 0.9765 - loss: 0.0938


[0.09376081824302673, 0.9764999747276306]

In [18]:
inputs = keras.Input(shape=((784,)))
x = layers.Dense(1024, activation="relu", name="first_layer")(inputs)
outputs = layers.Dense(10, activation="softmax")(x)
modelDFNN3 = keras.Model(inputs=inputs, outputs=outputs)

modelDFNN3.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

modelDFNN3.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
modelDFNN3.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 4s - 2ms/step - accuracy: 0.9443 - loss: 0.1841
Epoch 2/5
1875/1875 - 3s - 1ms/step - accuracy: 0.9772 - loss: 0.0745
Epoch 3/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9853 - loss: 0.0470
Epoch 4/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9890 - loss: 0.0341
Epoch 5/5
1875/1875 - 3s - 1ms/step - accuracy: 0.9917 - loss: 0.0252
313/313 - 1s - 4ms/step - accuracy: 0.9798 - loss: 0.0736


[0.07356487214565277, 0.9797999858856201]

In [19]:
inputs = keras.Input(shape=((784,)))
x = layers.Dense(1024, activation="relu", name="first_layer")(inputs)
x = layers.Dense(128, activation="relu", name="third_layer")(x)
outputs = layers.Dense(10, activation="softmax")(x)
modelDFNN3 = keras.Model(inputs=inputs, outputs=outputs)

modelDFNN3.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

modelDFNN3.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
modelDFNN3.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 5s - 2ms/step - accuracy: 0.9452 - loss: 0.1820
Epoch 2/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9754 - loss: 0.0777
Epoch 3/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9822 - loss: 0.0545
Epoch 4/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9867 - loss: 0.0415
Epoch 5/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9900 - loss: 0.0328
313/313 - 1s - 4ms/step - accuracy: 0.9818 - loss: 0.0668


[0.06675678491592407, 0.9818000197410583]